In [1]:
import numpy as np
import cv2

import rospy
import sensor_msgs
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

DIM=(2448, 2048)
K=np.array([[677.6729654637016, 0.0, 1235.371481335309], [0.0, 679.4736576804626, 1043.9887347932538], [0.0, 0.0, 1.0]])
D=np.array([[0.026055346132657364], [0.006035766757842894], [0.005666324884814231], [-0.0015661403498746557]])

PUCK_RADIUS=0.038
pixel_scale_x=2448.0/3.0
pixel_scale_y=2048.0/2.0
PUCK_RADIUS_pixel_x=pixel_scale_x*PUCK_RADIUS
PUCK_RADIUS_pixel_y=pixel_scale_y*PUCK_RADIUS

class Camera():
    def __init__(self, DIM, K, D):
        self.DIM = DIM
        self.K_camera = K
        self.K_projection = K
        self.D = D
        self.warp_matrix = np.eye(3, 3, dtype=np.float32)

    def undistort(self, img):
        undistorted_img = cv2.fisheye.undistortImage(distorted=img,
                                                     K=self.K_camera,
                                                     D=self.D,
                                                     Knew=self.K_projection)
        return undistorted_img
        
class CameraFilterNode():
    def __init__(self, DIM, K, D):
        self.node = rospy.init_node('camera_filter_node', anonymous=True)
        self.publisher = rospy.Publisher("/filtered_image", Image)
        self.subscriber = rospy.Subscriber("/usb_cam/image_raw", sensor_msgs.msg.Image, self.__callback)
        self.bridge = CvBridge()
        self.camera = Camera(DIM, K, D)
        self.it = 0


    def __callback(self, data):
        try:
            cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
            rospy.loginfo(rospy.get_caller_id())
        except CvBridgeError as e:
            print(e)
        
        #cv_image = cv2.medianBlur(cv_image,5)
        #cv_image = cv2.GaussianBlur(cv_image,(5,5),0)
        cv_image = cv2.bilateralFilter(cv_image,9,75,75)
#         cv_image_gray = cv2.cvtColor(cv_image, cv2.COLOR_RGB2GRAY)
#         cv_image = cv2.adaptiveThreshold(cv_image_gray
#                                          ,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv2.THRESH_BINARY,11,2)
        undistorted_image = self.camera.undistort(cv_image)
        image = undistorted_image
        image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        image1 = cv2.adaptiveThreshold(image_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        ret, thresh = cv2.threshold(image_gray, 127, 255, 0)
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            cnt = cv2.convexHull(cnt)
        cnts = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            
            if area == np.pi*PUCK_RADIUS_pixel_x*PUCK_RADIUS_pixel_y and len(cnt)>=5:
                print (area)
                cnts.append(cnt)
        im3 = image
        for cnt in cnts:   
            ellipse = cv2.fitEllipse(cnt)
            im3 = cv2.ellipse(im3,ellipse,(0,255,0),2)
#         for cnt in contours:
#             ellipse = cv2.fitEllipse(cnt)
#             print (cnt)
#             im3 = cv2.ellipse(im3,ellipse,(0,255,0),2)
#             if cv2.arcLength(cnt,False) <= 2*np.pi*PUCK_RADIUS:
#                 cnts.append(cnt)
            
        #im3 = cv2.drawContours(image, contours, -1, (255,0,0), 3)
        
        try:
#             field = cv2.imread("/home/alexey/Desktop/field.png")
#             field = cv2.resize(field, (2448,2048))
#             image1 = undistorted_image // 2 + field // 2
            self.publisher.publish(self.bridge.cv2_to_imgmsg(im3, "bgr8"))
        except CvBridgeError as e:
            print(e)


filter_node = CameraFilterNode(DIM, K, D)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:37: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.


[INFO] [1545154453.594953]: /camera_filter_node_8454_1545154453324
898.5
1216.5
8.5
6.5
5.0
1.5
2.5
71.5
81.0
146.0
22.5
3.0
24.0
12.0
2.0
10.5
9.0
24.0
593.0
42.5
255.5
15.0
5.5
12.5
89.5
3.5
40.5
56.0
900.5
286.5
5.5
11.5
31.5
48.5
4.5
0.0
5.0
107.0
17.5
41.0
1714.5
25.5
16.5
0.0
209.0
4.0
8.5
7.0
30.0
0.0
8.0
3.5
5.0
5.0
7.0
1.5
5.5
89.5
81.5
87.5
6.0
5.0
75.5
28.0
87.0
19.0
3.5
25.0
29.5
300.0
5.5
368.0
14.0
10.0
15.0
327.0
3710.0
11.5
0.5
3.0
2429.0
7.0
4.0
9.5
33.5
122.0
1.5
35.5
10.0
44.0
7.0
290.5
23.0
48.5
133.5
78.0
3.0
34.0
10.0
21.5
0.5
183.0
26.5
20.0
87.0
17.0
3030.0
0.5
233.5
12.0
6.0
78.5
66.5
23.0
4.0
16.0
69.5
149.0
5.5
364.5
0.5
24.0
18.0
700.5
4.0
172.5
81.5
8.5
19.0
5.0
84.5
113.5
11.5
73.5
80.5
54.0
793.0
6.0
17.5
8.5
11.0
1.0
41.0
5.0
58.5
1151.5
5.5
2.0
1.5
4.5
4.5
1.5
42.0
11.5
8.5
16.0
35.0
2.5
10.0
16.5
2.5
27.0
3.0
13.0
31.5
448.5
0.0
27.0
90.5
203.0
10.0
476.5
18.0
8.0
1.0
435.5
1119.0
0.5
31.5
6.0
1543.0
915.5
59.0
11.5
26.0
0.5
0.5
1.5
2.5
53.0
1.5
10.0
4

973.5
1208.0
21.0
60.0
67.5
209.0
4.0
15.0
20.0
12.5
0.0
9.0
612.5
31.0
261.0
53.0
22.5
30.0
34.5
4.0
100.5
46.0
899.5
278.0
4.0
56.5
22.5
2.5
37.5
46.5
9.0
24.5
7.0
3.0
5.5
96.0
7.5
1718.5
31.0
1.0
10.5
72.0
2.0
2.0
9.5
160.0
4.0
12.5
4.0
44.0
34.0
81.5
7.0
105.5
37.5
3.5
13.0
5.0
2.5
337.0
344.5
5.5
457.0
47.5
8.0
3510.0
15.5
31.5
111.0
7.0
16.0
7.0
2.0
341.5
4.0
292.0
28.0
90.5
4.0
35.5
131.0
18.5
1.0
14.5
21.5
54.0
2525.0
22.5
1.0
28.5
11.5
13.5
3150.0
71.5
14.0
116.5
68.0
15.5
46.5
43.5
592.5
220.0
4.0
84.5
6.5
61.0
399.5
3.0
5.0
54.5
22.0
176.5
42.5
84.5
83.5
4.0
486.0
92.5
71.0
53.0
6.5
61.0
9.5
13.5
1.0
11.5
17.0
1209.0
2.5
24.0
12.0
4.0
124.5
17.5
24.0
22.0
9.5
41.5
4.0
89.5
78.5
102.0
188.0
1.5
443.5
7.0
34.0
5.0
4.0
15.5
462.0
4.0
1066.0
48.0
1517.0
0.5
927.5
11.5
13.0
12.0
6.0
32.5
37.0
46.0
412.5
16.5
3.5
4.0
11.5
18.0
5.5
14.0
4.0
4.0
6.0
59.0
5.5
4.0
5.5
50.0
5.5
86.5
4.0
70.0
11.0
39.0
38.5
4.0
9.0
11.0
45.0
74.0
4.0
4.0
5.5
4.0
93.0
18.5
4.0
12.5
12.5
29.0
8.5
23.5
10.

395.0
865.5
0.5
5.0
13.5
24.0
77.5
94.0
101.5
41.0
16.0
33.0
12.5
604.5
334.0
17.5
59.0
4.0
39.5
6.0
2.0
174.5
4.0
73.5
53.0
893.5
0.0
1.5
51.5
2.5
88.5
3.5
27.0
13.5
106.5
4.0
114.0
0.0
1703.5
29.0
7.5
0.5
0.0
21.0
2.5
0.0
4.0
4.0
21.5
55.0
4.0
60.0
66.0
92.0
2.0
41.5
106.5
4.0
74.5
197.0
4.0
6.0
6.0
42.5
371.0
19.0
4.0
7.5
339.0
3.0
3205.5
16.5
27.5
6.0
15.0
1.0
45.0
7.5
280.5
2243.5
5.5
248.0
68.0
4.0
108.0
4.5
27.5
9.5
53.0
100.5
4.0
6.0
13.0
462.0
2996.5
36.5
9.0
6.5
22.0
128.0
1.0
64.0
36.0
22.0
110.5
58.0
1.5
1.0
52.5
208.0
5.5
38.5
13.5
76.5
6.0
174.0
4.0
4.0
6.0
176.0
5.5
31.5
45.0
497.5
100.5
68.0
85.0
43.0
4.0
70.5
1.5
28.5
12.5
968.5
5.5
221.0
5.0
3.0
56.0
1.5
5.0
53.5
19.0
17.0
8.0
39.0
9.5
31.5
164.5
93.0
434.0
18.0
437.0
569.5
4.0
284.0
4.0
4.0
1039.0
14.5
53.0
9.5
1504.5
22.5
929.5
11.5
2.0
24.0
6.5
4.5
15.0
2.0
5.5
4.0
4.0
7.0
7.5
4.0
36.0
4.0
57.0
12.5
21.5
99.0
9.5
4.0
86.0
4.0
6.0
4.0
35.5
5.5
344.5
6.0
4.0
4.0
70.0
5.5
4.0
4.0
88.0
116.5
4.0
39.5
38.0
20.5
30.5
49.

842.5
925.5
4.0
1.5
1.0
3.0
81.0
88.0
4.0
64.0
18.5
6.0
0.0
13.5
2.0
3.0
14.5
3.0
610.0
291.5
9.5
30.5
7.0
37.5
5.5
113.5
69.5
56.0
5.0
908.5
7.0
37.0
4.0
53.0
22.0
4.5
78.5
4.5
16.0
1.0
15.0
1726.5
29.0
64.0
1643.0
4.0
14.5
12.5
3.5
30.0
77.5
281.0
28.5
3.5
2.0
10.0
4.5
78.0
93.0
139.0
4.0
10.0
91.0
33.5
99.0
30.0
88.5
337.5
4.0
85.5
81.5
7.5
34.5
330.0
12.5
3531.5
49.5
1.0
3.0
53.0
672.5
3.5
3.5
290.5
3.0
6.5
173.5
6.0
8.0
1.0
32.5
1.0
6.0
131.0
1.0
18.5
8.5
86.5
5.5
16.0
31.0
37.5
3069.0
62.5
211.5
73.5
20.5
137.5
8.0
9.5
351.0
2.0
0.0
173.5
10.0
689.5
5.5
4.0
7.0
350.5
119.0
12.5
7.0
15.5
21.0
95.5
15.5
3.0
66.5
12.5
4.0
4.0
7.0
68.0
12.0
7.0
4.0
27.5
1182.0
3.0
14.0
24.0
7.0
16.0
3.5
38.5
380.0
4.0
7.0
4.0
56.0
4.0
5.5
1.5
2.5
2.5
21.0
8.0
78.0
6.0
27.5
183.5
5.5
31.0
95.0
548.0
6.0
457.0
3.5
6.5
38.5
17.0
4.0
398.0
4.0
1078.5
16.0
34.0
1571.5
27.0
939.0
6.5
31.5
25.5
7.0
1.5
17.0
128.5
30.5
434.5
3.5
2.0
5.5
4.0
4.0
14.0
5.5
5.5
31.5
303.5
4.0
5.5
5.5
68.0
4.0
5.5
30.5
48.5
4.0
4

668.0
746.5
98.0
110.5
7.5
32.5
4.0
20.0
10.5
1.0
608.5
35.0
280.5
1.5
16.5
71.0
8.5
44.0
909.0
65.5
23.5
50.5
30.5
9.0
5.0
43.5
8.0
1.0
35.0
1716.0
29.0
5.5
0.0
10.5
54.0
4.0
387.5
13.0
6.0
94.5
43.5
24.0
202.5
10.0
12.0
80.5
9.5
49.0
54.5
19.0
376.5
6.0
327.5
4.0
2.0
3296.5
97.5
19.0
14.0
25.0
48.0
1.0
6.0
292.0
99.5
2292.0
4.0
5.5
14.5
124.0
10.0
2.5
8.0
3.0
7.5
17.5
9.0
132.5
41.0
2984.0
5.5
4.0
73.5
1.0
5.0
10.0
55.5
24.5
9.5
34.5
550.0
4.0
128.5
1.0
4.5
6.5
39.0
208.0
8.0
0.5
172.0
636.5
4.0
29.5
45.5
89.0
4.0
83.5
10.0
42.0
4.0
42.0
3.0
3.0
1.0
1.0
6.5
38.5
31.5
5.0
1.0
1.5
22.5
10.5
922.0
0.0
28.5
15.5
13.0
23.0
2.0
17.0
15.0
57.5
77.0
1.5
10.5
22.0
97.0
116.0
3.0
7.5
513.5
427.0
7.5
35.0
33.0
1.5
395.0
1011.0
28.5
4.0
0.5
1501.5
10.0
10.0
947.0
0.0
16.5
5.5
4.0
10.0
27.5
20.5
3.5
3.5
4.0
24.5
281.5
21.5
9.0
85.0
17.0
1.5
37.0
21.5
12.5
5.0
6.0
28.0
5.5
10.0
4.0
6.0
36.0
10.0
4.0
7.0
202.0
4.0
10.0
5.5
32.5
38.5
4.0
20.5
4.0
20.0
10.0
4.0
5.5
25.0
4.0
4.0
5.5
4.0
45.0
7.0
4.0
1

In [1]:
import numpy as np
import cv2

import rospy
import sensor_msgs
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

DIM=(2448, 2048)
K=np.array([[677.6729654637016, 0.0, 1235.371481335309], [0.0, 679.4736576804626, 1043.9887347932538], [0.0, 0.0, 1.0]])
D=np.array([[0.026055346132657364], [0.006035766757842894], [0.005666324884814231], [-0.0015661403498746557]])

PUCK_RADIUS=0.038
pixel_scale_x=2448.0/3.0
pixel_scale_y=2048.0/2.0
PUCK_RADIUS_pixel_x=pixel_scale_x*PUCK_RADIUS
PUCK_RADIUS_pixel_y=pixel_scale_y*PUCK_RADIUS

class Camera():
    def __init__(self, DIM, K, D):
        self.DIM = DIM
        self.K_camera = K
        self.K_projection = K
        self.D = D
        self.warp_matrix = np.eye(3, 3, dtype=np.float32)

    def undistort(self, img):
        undistorted_img = cv2.fisheye.undistortImage(distorted=img,
                                                     K=self.K_camera,
                                                     D=self.D,
                                                     Knew=self.K_projection)
        return undistorted_img
        
class CameraFilterNode():
    def __init__(self, DIM, K, D):
        self.node = rospy.init_node('camera_filter_node', anonymous=True)
        self.publisher = rospy.Publisher("/filtered_image", Image)
        self.subscriber = rospy.Subscriber("/usb_cam/image_raw", sensor_msgs.msg.Image, self.__callback)
        self.bridge = CvBridge()
        self.camera = Camera(DIM, K, D)
        self.it = 0


    def __callback(self, data):
        try:
            cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
            rospy.loginfo(rospy.get_caller_id())
        except CvBridgeError as e:
            print(e)
        
        #cv_image = cv2.medianBlur(cv_image,5)
        #cv_image = cv2.GaussianBlur(cv_image,(5,5),0)
        cv_image = cv2.bilateralFilter(cv_image,9,75,75)
#         cv_image_gray = cv2.cvtColor(cv_image, cv2.COLOR_RGB2GRAY)
#         cv_image = cv2.adaptiveThreshold(cv_image_gray
#                                          ,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv2.THRESH_BINARY,11,2)
        undistorted_image = self.camera.undistort(cv_image)
        image = undistorted_image
        

        
        try:
            self.publisher.publish(self.bridge.cv2_to_imgmsg(image, "bgr8"))
        except CvBridgeError as e:
            print(e)


filter_node = CameraFilterNode(DIM, K, D)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:37: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.


[INFO] [1545231960.052910]: /camera_filter_node_19962_1545231959839
[INFO] [1545231960.598296]: /camera_filter_node_19962_1545231959839
[INFO] [1545231961.102119]: /camera_filter_node_19962_1545231959839
[INFO] [1545231961.604028]: /camera_filter_node_19962_1545231959839
[INFO] [1545231962.106205]: /camera_filter_node_19962_1545231959839
[INFO] [1545231962.594789]: /camera_filter_node_19962_1545231959839
[INFO] [1545231963.138793]: /camera_filter_node_19962_1545231959839
[INFO] [1545231963.658578]: /camera_filter_node_19962_1545231959839
[INFO] [1545231964.184959]: /camera_filter_node_19962_1545231959839
[INFO] [1545231964.697147]: /camera_filter_node_19962_1545231959839
[INFO] [1545231965.209236]: /camera_filter_node_19962_1545231959839
[INFO] [1545231965.723210]: /camera_filter_node_19962_1545231959839
[INFO] [1545231966.249047]: /camera_filter_node_19962_1545231959839
[INFO] [1545231966.753343]: /camera_filter_node_19962_1545231959839
[INFO] [1545231967.262982]: /camera_filter_node_